In [ ]:
bet1 = 2.4
bet2 = 1.59

Arb_percentage = 1/bet1 + 1/bet2
print("Arb_per:  {0:.3f}".format(Arb_percentage))

win_amount = 50

total_outlay = (win_amount/bet1) + (win_amount/bet2)

profit = win_amount - total_outlay
roi = profit/total_outlay


print("Total_outlay:  {0:.2f}".format(total_outlay))
print("Bet on 1st: '  {0:.2f}".format(win_amount/bet1))
print("Bet on 2nd: '  {0:.2f}".format(win_amount/bet2))
print("Profit:        {0:.2f}".format(profit))
print("ROI:           {0:.2f}".format(roi))